<a href="https://colab.research.google.com/github/milicaci/obuznie/blob/master/DZ_BD_Itog1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

1. Загрузили в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы.

In [2]:
df_m = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.item', sep= '|', header = None, encoding='latin-1')
#df_m

In [3]:
df_m.columns = ['item id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
df_m.head(5)

,item id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [4]:
df_r = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', sep= '\t', header = None, encoding='latin-1')
#df_r

In [5]:
df_r.columns = ['user_id', 'item id', 'rating', 'timestamp']
df_r.head(5)

,user_id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
#df_r.info()

2.Найден id пользователя, поставившего больше всего оценок.


In [7]:
df_r[['user_id', 'rating']].groupby('user_id').count().sort_values('rating', ascending = False).head(1)

,rating
user_id,
405,737


3. Оставили в датафрейме ratings только те фильмы, который оценил  пользователь 405.


In [8]:
df_ra = df_r[df_r['user_id'] == 405]
df_ra

,user_id,item id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447
...,...,...,...,...
98956,405,375,1,885546835
98978,405,445,4,885548435
99148,405,1246,1,885547735
99465,405,196,1,885546112


4. Добавили к датафрейму из задания 3 столбцы:
*   По жанрам.( Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей)
*   Общее количество оценок
*   Суммарную оценку





In [9]:
df_m['year'] = pd.to_datetime(df_m['release date']).dt.strftime('%Y')

In [10]:
df_mab = df_m[[ 'movie title', 'year', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir', 'Horror',  'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western', 'item id']]

In [11]:
df_ra['Sumarnaya oshenka'] = df_ra[['item id', 'rating']].groupby(['item id'])['rating'].transform('sum')
df_ra['Obchee kol oshenok'] = df_ra[['item id', 'rating']].groupby(['item id'])['rating'].transform('count')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
result = pd.merge(df_ra, df_mab, how = 'left', on = 'item id')
result.head(5)

,user_id,item id,rating,timestamp,Sumarnaya oshenka,Obchee kol oshenok,movie title,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,405,56,4,885544911,4,1,Pulp Fiction (1994),1994,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,1,1,True Crime (1995),1995,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,885548670,1,1,T-Men (1947),1947,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,405,171,1,885549544,1,1,Delicatessen (1991),1991,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,405,580,1,885547447,1,1,"Englishman Who Went Up a Hill, But Came Down a...",1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [13]:
result.columns

Index(['user_id', 'item id', 'rating', 'timestamp', 'Sumarnaya oshenka',
       'Obchee kol oshenok', 'movie title', 'year', 'unknown', 'Action',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

5. Сформировали X_train, X_test, y_train, y_test

In [14]:
X = result[['Sumarnaya oshenka', 'Obchee kol oshenok', 'year', 'unknown', 'Action',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']]
y = result['rating']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
X_train

,Sumarnaya oshenka,Obchee kol oshenok,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
633,2,1,1992,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
118,1,1,1994,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
310,1,1,1992,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
136,1,1,1986,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
167,1,1,1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,1,1,1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
34,1,1,1986,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0
735,1,1,1989,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
427,1,1,1994,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0


In [17]:
X_test

,Sumarnaya oshenka,Obchee kol oshenok,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
275,2,1,1990,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
707,1,1,1991,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
319,1,1,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
297,3,1,1994,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
375,1,1,1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,5,1,1981,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
553,1,1,1946,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
315,1,1,1993,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
584,1,1,1984,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [18]:
X_train = pd.DataFrame(X_train, columns = X.columns)
X_train.head(2)

,Sumarnaya oshenka,Obchee kol oshenok,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
633,2,1,1992,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
118,1,1,1994,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0


In [19]:
X_test = pd.DataFrame(X_test, columns=X.columns)
X_test.head(2)

,Sumarnaya oshenka,Obchee kol oshenok,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
275,2,1,1990,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
707,1,1,1991,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


6 Взяли модель линейной регрессии (или любую другую для задачи регрессии) и обучите ее на фильмах.

In [20]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
linear = LinearRegression()
linear.fit(X_train, y_train)
linear.predict(X_test)

array([2., 1., 1., 3., 1., 1., 1., 1., 2., 1., 1., 5., 4., 1., 1., 4., 1.,
       1., 1., 3., 1., 4., 1., 1., 1., 1., 5., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 5., 2., 1., 1., 1., 1., 3., 2., 2., 1., 1.,
       1., 1., 1., 4., 1., 1., 5., 1., 1., 1., 1., 1., 1., 2., 2., 4., 3.,
       1., 1., 1., 1., 5., 2., 4., 3., 1., 1., 1., 5., 3., 1., 1., 2., 1.,
       1., 1., 1., 1., 1., 1., 5., 2., 1., 1., 5., 1., 1., 1., 5., 3., 1.,
       1., 1., 1., 2., 1., 1., 5., 5., 1., 5., 1., 1., 1., 1., 2., 1., 3.,
       2., 1., 5., 1., 1., 1., 1., 1., 3., 1., 5., 1., 3., 2., 5., 1., 3.,
       1., 1., 1., 1., 2., 1., 1., 5., 1., 1., 1., 3.])

7. Оценили качество модели на X_test, y_test при помощи метрик для задачи регресси

In [22]:
mean_squared_error(y_test, linear.predict(X_test))# среднеквадратичная ошибка

4.265612103421641e-30

In [23]:
r2_score(y_test, linear.predict(X_test)) # коэффициент детерминации

1.0

In [24]:
mean_absolute_error(y_test, linear.predict(X_test)) #  средняя абсолютная ошибка 

1.566314645552248e-15